**In this Notebook I will apply basic NLP techniques which includes regular expressions to remove unwanted characters from a text, lammetizaion,stemming , removing stop words, part of speech tagging and visualisation and understanding patterns**

Some Basics:
    Regular Expressions:
    A regular expression (sometimes called a rational expression) is a sequence of characters that define a search pattern, mainly for use in pattern matching with strings, or string matching, i.e. “find and replace”-like operations.(Wikipedia).

Regular expressions are a generalized way to match patterns with sequences of characters. It is used in every programming language like C++, Java and Python.

What is a regular expression and what makes it so important?
Regex are used in Google analytics in URL matching in supporting search and replace in most popular editors like Sublime, Notepad++, Brackets, Google Docs and Microsoft word.



Example :  Regular expression for an email address :
^([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})$ 

How to write regular expression ?
For that you need to learn various rule for the same. 
You may refer below link:
   [NTL by Dan Jurafsky](http://youtu.be/3Dt_yh1mf_U)
   
 POS tagging:
 It is used to identify the part of speech of a word in text. NLTK povide methods for pos tagging. A comprehancive list of POS tags short list is as below:
  will use POS tagging ( which is used for tagging a word for its part of speech as per engilsh grammer) for filtering words which can be used as features for classification of tweets


Stop words:
it , the , a etc words generaly dont help in finding sentiments hence these are generaly being removed from the text. These are called stop words. 

Please refer the link given above to undestand basics of natural language programming




In [ ]:
#POS tagging code
'''CC coordinating conjunction
CD cardinal digit
DT determiner
EX existential there (like: "there is" ... think of it like "there exists")
FW foreign word
IN preposition/subordinating conjunction
JJ adjective 'big'
JJR adjective, comparative 'bigger'
JJS adjective, superlative 'biggest'
LS list marker 1)
MD modal could, will
NN noun, singular 'desk'
NNS noun plural 'desks'
NNP proper noun, singular 'Harrison'
NNPS proper noun, plural 'Americans'
PDT predeterminer 'all the kids'
POS possessive ending parent's
PRP personal pronoun I, he, she
PRP$ possessive pronoun my, his, hers
RB adverb very, silently,
RBR adverb, comparative better
RBS adverb, superlative best
RP particle give up
TO to go 'to' the store.
UH interjection errrrrrrrm
VB verb, base form take
VBD verb, past tense took
VBG verb, gerund/present participle taking
VBN verb, past participle taken
VBP verb, sing. present, non-3d take
VBZ verb, 3rd person sing. present takes
WDT wh-determiner which
WP wh-pronoun who, what
WP$ possessive wh-pronoun whose
WRB wh-abverb where, when'''



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# General Libraries
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re
# specific for data preproressing and visualization
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS 
from statistics import mode
# classifiers
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC,NuSVC
from nltk.classify import ClassifierI

In [ ]:
TweetData =  pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")

In [ ]:
TweetData.head()

We have total 27486 rows but text and selected text is missing for one row Lets check the same

In [ ]:
TweetData.info()

In [ ]:
print(TweetData.isna().any())
TweetData[TweetData.isnull().any(axis=1)]

Lets Remove the Empty Text row

In [ ]:
TweetData.dropna(inplace=True)

In [ ]:
TweetData.info()

Lets see the Distributions of Sentiments

In [ ]:
# the count of sentiments in absolute value as we as percentage form

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
sns.countplot(TweetData['sentiment'], facecolor=(0, 0, 0, 0),
                   linewidth=5,
                   edgecolor=sns.color_palette("muted", 3))
plt.subplot(1,2,2)
(TweetData['sentiment'].value_counts()/len(TweetData)*100).plot(kind="bar", rot=0)

In [ ]:
# lets count the words in each tweet and add columns for it
def wordcount(text):
    words = word_tokenize(text)
    return len(words)

TweetData['Text_words'] = TweetData['text'].apply(lambda x : wordcount(x))
TweetData['Select_text_words']= TweetData['selected_text'].apply(lambda x : wordcount(x))

In [ ]:
TweetData.head()

In [ ]:
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
plt.figure(figsize=(12,6),dpi=100)
plt.subplot(1,2,1)
sns.distplot(TweetData['Text_words'],kde=False)
plt.subplot(1,2,2)
sns.distplot(TweetData['Select_text_words'],kde=False)

Majority of text have length 1-10 and it has been reduced to 1 to 3 or 4 length word as we can se above It might helpful while applying model in test data

Lets Clean the tweets using Regular Expressions

In [ ]:
stop_words = set(stopwords.words('english'))
def clean_tweets(x):
    # removing the hyperlinks
    clean1 = re.sub('https?://[A-Za-z0-9./]+','',x)
    # removing the hashtags
    clean2 = re.sub('#[A-Za-z0-9]+','',clean1)
    # removing @
    clean3 = re.sub('@[A-Za-z0-9]','',clean2)
    # removing punctuations and lower case conversion
    clean4 = re.sub(r'[^\w\s]','',clean3).lower()
    words = word_tokenize(clean4)
    # removing stopwords
    words = [w for w in words if not w in stop_words]
    sent = ' '.join(words)
    return sent

In [ ]:
# lets first check it for few sentences:
for i in range(12,30):
    Clean = clean_tweets(TweetData['text'][i])
    print("Text: ",TweetData['text'][i])
    print("Clean Text: ", Clean)

In [ ]:
# lets apply our clean tweet funcion for all tests
TweetData['Clean_tweet'] = TweetData['text'].apply(lambda x : clean_tweets(x))
TweetData['Clean_tweet_words'] = TweetData['Clean_tweet'].apply(lambda x : wordcount(x))

In [ ]:
TweetData.head()

In [ ]:
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=1.2)
plt.figure(figsize=(12,6),dpi=100)
plt.subplot(1,3,1)
sns.distplot(TweetData['Text_words'],kde=False)
plt.subplot(1,3,2)
sns.distplot(TweetData['Select_text_words'],kde=False)
plt.subplot(1,3,3)
sns.distplot(TweetData['Clean_tweet_words'],kde=False)

# Lets see the words Frequency Distribution

In [ ]:
# All word Freqency curve with out any pos tagging filter
All_words = []
for sent in TweetData['Clean_tweet']:
    for word in word_tokenize(sent):
            All_words.append(word)
All_words_freq = nltk.FreqDist(All_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
sns.barplot('Words','freq',data = Freq_word_DF)

Lets See the Positive , Neutral and Negative tweets words Frequency Distribution Seperatly 

In [ ]:
# TweetData[TweetData['sentiment']=='positive']['Clean_tweet']
All_words = []
for sent in TweetData[TweetData['sentiment']=='positive']['Clean_tweet']:
    for word in word_tokenize(sent):
            All_words.append(word)
All_words_freq = nltk.FreqDist(All_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
ax = sns.barplot('Words','freq',data = Freq_word_DF)
ax.set_title("Positive Tweets Words Frequency Plot")

# Negative tweets
All_words = []
for sent in TweetData[TweetData['sentiment']=='negative']['Clean_tweet']:
    for word in word_tokenize(sent):
            All_words.append(word)
All_words_freq = nltk.FreqDist(All_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
ax = sns.barplot('Words','freq',data = Freq_word_DF)
ax.set_title("Negative Tweets Words Frequency Plot")

# Neutral Tweets

All_words = []
for sent in TweetData[TweetData['sentiment']=='neutral']['Clean_tweet']:
    for word in word_tokenize(sent):
            All_words.append(word)
All_words_freq = nltk.FreqDist(All_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
ax = sns.barplot('Words','freq',data = Freq_word_DF)
ax.set_title("Neutral Tweets Words Frequency Plot")

# Observations:
**We can see that positive tweets common word contain adjectives e.g Love Good Happy Thanks Hope
and Negative tweets have common words such as dont cant sad sorry etc
while neutral has words like get go ad nothing from positive or negative words like happy, sad good etc**

This seems significant deviaion and if we can make feature of each words of these 15*3 words we can train a model for the same. 
But our job is not to make a model to predict the sentiment of the same. Our job is to predict which part of text is creating the sentiment.

I will come to it but lets see how further we can explore the texts 

In [ ]:
# Lets do POS tagging of our clean tweets
def tagging(text):
    tag = nltk.pos_tag(word_tokenize(text))
    return tag

In [ ]:
TweetData['Pos_Clean'] = TweetData['Clean_tweet'].apply(lambda x : tagging(x))

In [ ]:
TweetData[['Clean_tweet','Pos_Clean']].head()

In [ ]:
# Lets See Frequency Distribution of just Adjectives in clean tweet from positive negative and neutal tweets
allowed_word_type = ["JJ","JJR","JJS"]
all_words = []
for pos in TweetData['Pos_Clean']:
    for w in pos:
        if w[1] in allowed_word_type:
            all_words.append(w[0])
All_words_freq = nltk.FreqDist(all_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
ax = sns.barplot('Words','freq',data = Freq_word_DF)
ax.set_title("Adjective words frequncy distribution")

In [ ]:
# The code worked above lets check for all types of setiment words:
all_words = []
allowed_word_type = ["JJ","JJR","JJS"]
for pos in TweetData[TweetData['sentiment']=='positive']['Pos_Clean']:
    for w in pos:
        if w[1] in allowed_word_type:
            all_words.append(w[0])
All_words_freq = nltk.FreqDist(all_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
ax = sns.barplot('Words','freq',data = Freq_word_DF)
ax.set_title("Adjective words frequncy distribution for Positive Tweets")

# Negative words
for pos in TweetData[TweetData['sentiment']=='negative']['Pos_Clean']:
    for w in pos:
        if w[1] in allowed_word_type:
            all_words.append(w[0])
All_words_freq = nltk.FreqDist(all_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
ax = sns.barplot('Words','freq',data = Freq_word_DF)
ax.set_title("Adjective words frequncy distribution for Negative Tweets")

# neutral words
for pos in TweetData[TweetData['sentiment']=='neutral']['Pos_Clean']:
    for w in pos:
        if w[1] in allowed_word_type:
            all_words.append(w[0])
All_words_freq = nltk.FreqDist(all_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
ax = sns.barplot('Words','freq',data = Freq_word_DF)
ax.set_title("Adjective words frequncy distribution for Neutral Tweets")

In [ ]:
TweetData[TweetData['sentiment']=='negative'].head()

# Observations:
This adjective process does not provide great results 
what seems to me that for positive tweets adjective have great impact , for negative the verbs like dont,cant , and some adjactive like sad, sorry are differanciating 
Therefore in our analysis we will take this into our account when designing the model


In [ ]:
# Lets make some word clouds:
from wordcloud import WordCloud,STOPWORDS 
stopwords = set(STOPWORDS) 
comment_words = ' '
for text in TweetData['Clean_tweet']:
    for words in word_tokenize(text): 
        comment_words = comment_words + words + ' '
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words)
sns.set()
plt.figure(figsize = (8, 8), facecolor = None,dpi=100) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0)

In [ ]:
# Lets make some word clouds for positive Negative and Neural text:
from wordcloud import WordCloud,STOPWORDS 
stopwords = set(STOPWORDS) 
comment_words = ' '
for text in TweetData[TweetData['sentiment']=='positive']['Clean_tweet']:
    for words in word_tokenize(text): 
        comment_words = comment_words + words + ' '
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words)
sns.set()
plt.figure(figsize = (8, 8), facecolor = None,dpi=100)
plt.title("Positive Tweets word Cloud")
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0)


comment_words = ' '
for text in TweetData[TweetData['sentiment']=='negative']['Clean_tweet']:
    for words in word_tokenize(text): 
        comment_words = comment_words + words + ' '
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words)
sns.set()
plt.figure(figsize = (8, 8), facecolor = None,dpi=100)
plt.title("Negative Tweets word Cloud")
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0)


comment_words = ' '
for text in TweetData[TweetData['sentiment']=='neutral']['Clean_tweet']:
    for words in word_tokenize(text): 
        comment_words = comment_words + words + ' '
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words)
sns.set()
plt.figure(figsize = (8, 8), facecolor = None,dpi=100)
plt.title("Neutral Tweets word Cloud")
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0)



In [ ]:
# **Looking into the Selected Tweets**

In [ ]:
# Cleaning the Selected Tweets:
# stop_words = set(stopwords.words('english'))
def clean_tweets_selected(x):
    # removing the hyperlinks
    clean1 = re.sub('https?://[A-Za-z0-9./]+','',x)
    # removing the hashtags
    clean2 = re.sub('#[A-Za-z0-9]+','',clean1)
    # removing @
    clean3 = re.sub('@[A-Za-z0-9]','',clean2)
    # removing punctuations and lower case conversion
    clean4 = re.sub(r'[^\w\s]','',clean3).lower()
    words = word_tokenize(clean4)
    # removing stopwords
    words = [w for w in words if not w in stop_words]
    sent = ' '.join(words)
    return sent

In [ ]:
TweetData['Clean_Selected_Tweets'] = TweetData['selected_text'].apply(lambda x: clean_tweets_selected(x))

In [ ]:
TweetData.head()

In [ ]:
# All Selected word Freqency curve with out any pos tagging filter
All_words = []
for sent in TweetData['Clean_Selected_Tweets']:
    for word in word_tokenize(sent):
        All_words.append(word)
All_words_freq = nltk.FreqDist(All_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
sns.barplot('Words','freq',data = Freq_word_DF)

In [ ]:
# Lets See only selected words with one and two words only
All_words = []
for sent in TweetData['selected_text']:
    if len(word_tokenize(sent))<3:
        for word in word_tokenize(sent):
            All_words.append(word)
All_words_freq = nltk.FreqDist(All_words)
Freq_word_DF = pd.DataFrame({"Data":All_words_freq.most_common(15)})
Freq_word_DF['Words'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=2)
fig=plt.figure(figsize =(20,8),dpi=50)
sns.barplot('Words','freq',data = Freq_word_DF)

Observations:
Most of the single and double letter words are adjectives lets do POS tagging these small selected text and check pos distributions

In [ ]:
TweetData['Pos_selected_clean'] = TweetData['Clean_Selected_Tweets'].apply(lambda x : tagging(x))

In [ ]:
TweetData.head()

In [ ]:
Pos = []
for pos in TweetData['Pos_selected_clean']:
    for po in pos:
        Pos.append(po[1])

plt.figure(figsize=(12,6),dpi=100)
sns.set_context(font_scale=.5)
sns.countplot(Pos, facecolor=(0, 0, 0, 0),
                   linewidth=2,
                   edgecolor=sns.color_palette("muted", 2))        

plt.xticks(rotation=70)        

**The count of selected text with less than 3 words and check the pos for these texts**


In [ ]:
print("number of text with less than 3 selected text length: ",len(TweetData[TweetData['Select_text_words'] < 3]))
Pos = []
for pos in TweetData[TweetData['Select_text_words'] < 3]['Pos_selected_clean']:
    for po in pos:
        Pos.append(po[1])

plt.figure(figsize=(12,6),dpi=100)
sns.set_context(font_scale=.5)
sns.countplot(Pos, facecolor=(0, 0, 0, 0),
                   linewidth=2,
                   edgecolor=sns.color_palette("muted", 2))        

plt.xticks(rotation=70)      

# Observations:
**Most of the selected text are either adjective or common noun**
**If we look into the selected text of two and less its dominated by singular common noun or adjective**

Next I will do one check : I will select NN and NNS and JJ from clean text and same from the selected clean text, then I will check the differnece in number of these .. if its near to zero means all or most of the NN and NNS and JJ are in selected text.. which will give me guidance where to go furher 

In [ ]:
TweetData.head()

In [ ]:
selected_pos =  ["JJ","NN","NNS"]
def pos_filter(text):
    con = 0
    for w in text:
        if w[1] in selected_pos:
            con+= 1
    return con

In [ ]:
TweetData['count_text'] = TweetData['Pos_Clean'].apply(lambda x: pos_filter(x))
TweetData['Count_selected_text'] = TweetData['Pos_selected_clean'].apply(lambda x: pos_filter(x))

In [ ]:
TweetData.head()

In [ ]:
# Line plot for both counts above for different tweet sentiments , also for those which has only less that three words in selected text



In [ ]:
len(TweetData)

In [ ]:
plt.figure(figsize=(12,6),dpi=100)
sns.set_context('paper',font_scale=1)
sns.lineplot(x=TweetData.index[0:50],y = TweetData['count_text'][0:50])
sns.lineplot(x=TweetData.index[0:50],y = TweetData['Count_selected_text'][0:50])

In [ ]:
TweetDatasmall =  TweetData[TweetData['Select_text_words'] < 3]
plt.figure(figsize=(12,6),dpi=100)
sns.set_context('paper',font_scale=1)
sns.lineplot(x=TweetDatasmall.index[0:50],y = TweetDatasmall['count_text'][0:50])
sns.lineplot(x=TweetDatasmall.index[0:50],y = TweetDatasmall['Count_selected_text'][0:50])

Even for less than 3 words in selected text we can see significant gap.. perhaps we can apply double fitler one is finding adjective and nouns and filtering via most frequent words 

In [ ]:
# live curve for less that 3 words seggregated for difference sentiments :
Positive =  TweetDatasmall[TweetDatasmall['sentiment']=='positive']
Negative =  TweetDatasmall[TweetDatasmall['sentiment']=='negative']
Neutral =   TweetDatasmall[TweetDatasmall['sentiment']=='neutral']

plt.figure(figsize=(12,12),dpi=100)
plt.subplot(3,1,1)
sns.set_context('paper',font_scale=1)
sns.lineplot(x=Positive.index[0:50],y = Positive['count_text'][0:50])
sns.lineplot(x=Positive.index[0:50],y = Positive['Count_selected_text'][0:50])
plt.title("Positive tweets count Line Plots")
plt.subplot(3,1,2)
sns.set_context('paper',font_scale=1)
sns.lineplot(x=Negative.index[0:50],y = Negative['count_text'][0:50])
sns.lineplot(x=Negative.index[0:50],y = Negative['Count_selected_text'][0:50])
plt.title("Negative tweets count Line Plots")
plt.subplot(3,1,3)
sns.set_context('paper',font_scale=1)
sns.lineplot(x=Neutral.index[0:50],y = Neutral['count_text'][0:50])
sns.lineplot(x=Neutral.index[0:50],y = Neutral['Count_selected_text'][0:50])
plt.title("Neutral tweets count Line Plots")

**Looking at N Grams **

In [ ]:
# For N grams first I would like to Clean the tweets with out removing stop words
def clean_tweets_N(x):
    # removing the hyperlinks
    clean1 = re.sub('https?://[A-Za-z0-9./]+','',x)
    # removing the hashtags
    clean2 = re.sub('#[A-Za-z0-9]+','',clean1)
    # removing @
    clean3 = re.sub('@[A-Za-z0-9]','',clean2)
    # removing punctuations and lower case conversion
    clean4 = re.sub(r'[^\w\s]','',clean3).lower()
#     words = word_tokenize(clean4)
#     # removing stopwords
#     words = [w for w in words if not w in stop_words]
#     sent = ' '.join(words)
    return clean4

In [ ]:
TweetData['Clean_for_N-gram'] = TweetData['text'].apply(lambda x : clean_tweets_N(x))

In [ ]:
from nltk.util import ngrams
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [ ]:
TweetData['Ngram-2'] = TweetData['Clean_for_N-gram'].apply(lambda x : extract_ngrams(x,2))
TweetData['Ngram-3'] = TweetData['Clean_for_N-gram'].apply(lambda x : extract_ngrams(x,3))

In [ ]:
TweetData.head()

In [ ]:
# Plotting Ngram-2
All_ngram = []
for sent in TweetData['Ngram-2']:
    for gram in sent:
        All_ngram.append(gram)
All_ngram_freq = nltk.FreqDist(All_ngram)
Freq_word_DF = pd.DataFrame({"Data":All_ngram_freq.most_common(15)})
Freq_word_DF['2-Grams'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
# sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=1)
fig=plt.figure(figsize =(12,8),dpi=100)
sns.barplot('2-Grams','freq',data = Freq_word_DF,dodge=False)

In [ ]:
# Plotting Ngram-3
All_ngram = []
for sent in TweetData['Ngram-3']:
    for gram in sent:
        All_ngram.append(gram)
All_ngram_freq = nltk.FreqDist(All_ngram)
Freq_word_DF = pd.DataFrame({"Data":All_ngram_freq.most_common(15)})
Freq_word_DF['3-Grams'] = Freq_word_DF['Data'].apply(lambda x : x[0])
Freq_word_DF['freq'] = Freq_word_DF['Data'].apply(lambda x : x[1])
sns.set()
# sns.set(style='whitegrid', rc={"grid.linewidth": 0.2})
sns.set_context("paper", font_scale=1)
fig=plt.figure(figsize =(12,8),dpi=100)
sns.barplot('3-Grams','freq',data = Freq_word_DF,dodge=False)
plt.xticks(rotation=70)  